In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
print(torch.__version__)

In [ ]:
!pip install pytorch_lightning==1.3.3
!pip install transformers==4.6.1

In [ ]:
import sys
sys.path.append("..")
import pandas as pd
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader

from input.hidden.src.data import load_data
from input.hidden.src.data import SodicDataset
from input.hidden.src.model import BertFinetuner

def run():
    pl.seed_everything(42)
    train_data, valid_data, test_data = load_data("../input/hidden/data/train.csv", "../input/hidden/data/test.csv")
    
    train_set = SodicDataset(train_data)
    valid_set = SodicDataset(valid_data)

    train_loader = DataLoader(train_set, batch_size=16, shuffle=True, num_workers=4, collate_fn=train_set.collate_fn)
    valid_loader = DataLoader(valid_set, batch_size=16, shuffle=False, num_workers=4, collate_fn=valid_set.collate_fn)

    model = BertFinetuner()
    trainer = pl.Trainer(
        default_root_dir="./",
        max_epochs=1,
        gpus=1,
        deterministic=True
    )
    trainer.fit(model, train_loader, valid_loader)

    test_set = SodicDataset(test_data)
    test_loader = DataLoader(test_set, batch_size=8, shuffle=False, num_workers=4, collate_fn=test_set.collate_fn)
    preds = trainer.predict(model, test_loader)
    preds = torch.cat(preds, dim=0).detach().tolist()

    test_df = pd.read_csv("../input/hidden/data/test.csv")
    test_df["label"] = preds
    result_df = test_df[["id", "label"]]
    result_df.to_csv("submission.csv", index=False)


if __name__ == "__main__":
    run()
